# Location modeling for microbiome colonization

$$
\begin{align}
\min_{v^k,y^k} \quad & -\sum_k \sum_j c_k^T v^k_j \\
\mathrm{s.t.}  \quad & S^k v^k_j = 0, \quad j \in \mathrm{Locations} \\
                     & l^k_j y^k_j \leq v^k_j \leq u^k_j y^k_j \\
                     & \sum_j y^k_j \leq X^k_0 + X^k_0 \mu^k T_c \\                     
                     & \sum_k y^k_j \leq 1, \quad j\in \mathrm{Locations} \\
                     & v^k_{lj} \geq \sum_i \frac{v^\mathrm{tot}_{lij}}{d_{ij}} + \sum_{p\neq j} \frac{v^\mathrm{synth}_{lp}}{d_{jp}}, \quad l\in \mathrm{Nutrients},\ j\in \mathrm{Locations},\ k\in \mathrm{Organisms} \\
                     & v^\mathrm{tot}_{il} \leq \sum_k \sum_j v^k_{lij} d_{ij}, \quad l\in \mathrm{Nutrients},\ i\in \mathrm{Sources} \\
                     & y^k_j \in \{0,1\}.
\end{align}
$$

## Recapitulate:
http://www.pnas.org/content/early/2017/10/04/1711596114.short
- macro-scale patterns:
    - [Fig 2A,B](http://www.pnas.org/content/pnas/early/2017/10/04/1711596114/F4.large.jpg)
- microhabitats composition (dominant):
    - dominant species: Bacteroides dominate all microhabitats
        - Bacteroides: 43% of densely colonized and 46% of sparsely colonized fields
        - C. aerofaciens: 11% in densely, 6% in sparsely
        - C. scindens: 8% in both
        - R. torques: 3% in dense, 2% in sparse    
- inter-taxa correlations ([Fig. S3](http://www.pnas.org/content/pnas/early/2017/10/04/1711596114/F9.large.jpg))
    - some species show negative correlation
    - but R^2 small and p-values not reported!
- micro-scale spatial patterns (numbers are for mouse)
    - proximal colon shows homogenized community due to mixing and dispersal by host factors
    - turnover for mucus is 6 h for mucus in goblet cells and 1 h for inner mucus layer in distal colon
    - replication time of B. thetaiotaomicron and E. coli is 3 h in mucus layer and 3-8 h in colonic contents
    - therefore, microbes divide only a few rounds before being shed along with mucus into lumen
    - gut contents traverse mouse intestine in 4-6 h
    - overall, the turnover and shedding diminishes/disrupts segregated communities and spatial structure
    - note: human dental plaque form clustered and ordered arrangements

### Bacteria:
http://www.pnas.org/content/early/2017/10/04/1711596114.short
- 6 species (in [AGORA](https://vmh.uni.lu/#downloadview)?):
    - Bacteroides cellulosilyticus (yes, DSM 14838)
    - Bacteroides caccae (yes, ATCC 43185)
    - Parabacteroides distasonis (yes, ATCC 8503)
    - Ruminococcus torques (yes, ATCC 27756, L2-14)
    - Clostridium scindens (yes, ATCC 35704)
    - Collinsella aerofaciens (yes, ATCC 25986)
- 4 Moderately abundant Bacteroides:
    - Bacteroides thetaiotaomicron (yes, VPI-5482)
    - Bacteroides vulgatus (yes, ATCC 8482)
    - Bacteroides ovatus (yes, ATCC 8483, SD CC 2a, SD CMC 3f)
    - Bacteroides uniformis (yes, ATCC 8492)
- 5 low abundance Firmicutes
    - Eubacterium rectale (yes, ATCC 33656, M104/1)
    - Clostridium spiroforme (yes, DSM 1552)
    - Faecalibacterium prausnitzii ()yes, L2-6
    - Ruminococcus obeum (yes, A2-162)
    - Dorea longicatena (yes, DSM 13814)

## Grid

In [1]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [2]:
from cobra.io.sbml3 import read_sbml_model

def repair_model(mdl):
    obj0 = mdl.optimize().f
    for rxn in mdl.reactions:
        rxn.id = rxn.id.replace('__40__','_').replace('__41__','')
    for met in mdl.metabolites:
        met.id = met.id.replace('__91__','_').replace('__93__','')
    mdl.repair()
    obj1 = mdl.optimize().f
    print('mu: before=%s. after=%s' % (obj0,obj1))
    assert abs(obj1-obj0)<1e-9

mdl_files = [
    # 6 species:
    # Bacteroides cellulosilyticus (yes, DSM 14838)
    'Bacteroides_cellulosilyticus_DSM_14838',
    # Bacteroides caccae (yes, ATCC 43185)
    'Bacteroides_caccae_ATCC_43185',
    # Parabacteroides distasonis (yes, ATCC 8503)
    'Parabacteroides_distasonis_ATCC_8503',
    # Ruminococcus torques (yes, ATCC 27756, L2-14)
    'Ruminococcus_torques_ATCC_27756',
    # Clostridium scindens (yes, ATCC 35704)
    'Clostridium_scindens_ATCC_35704',
    # Collinsella aerofaciens (yes, ATCC 25986)
    'Collinsella_aerofaciens_ATCC_25986',
    # 4 Moderately abundant Bacteroides:
    # Bacteroides thetaiotaomicron (yes, VPI-5482)
    'Bacteroides_thetaiotaomicron_VPI_5482',
    # Bacteroides vulgatus (yes, ATCC 8482)
    'Bacteroides_vulgatus_ATCC_8482',
    # Bacteroides ovatus (yes, ATCC 8483, SD CC 2a, SD CMC 3f)
    'Bacteroides_ovatus_ATCC_8483',
    # Bacteroides uniformis (yes, ATCC 8492)
    'Bacteroides_uniformis_ATCC_8492',
    # 5 low abundance Firmicutes
    # Eubacterium rectale (yes, ATCC 33656, M104/1)
    'Eubacterium_rectale_ATCC_33656',
    # Clostridium spiroforme (yes, DSM 1552)
    'Clostridium_spiroforme_DSM_1552',
    # Faecalibacterium prausnitzii ()yes, L2-6
    'Faecalibacterium_prausnitzii_L2_6',
    # Ruminococcus obeum (yes, A2-162)
    'Ruminococcus_obeum_A2_162',
    # Dorea longicatena (yes, DSM 13814)
    'Dorea_longicatena_DSM_13814'
]

from os import path

models = []
for fn in mdl_files:
    mdl = read_sbml_model(path.join('/home/laurence/ME/data/microbiome/Agora_1_02_Western/sbml', fn+'.xml'))
    repair_model(mdl)
    models.append(mdl)

    # B_caccae = read_sbml_model('/home/laurence/ME/data/microbiome/Agora_1_02_Western/sbml/Bacteroides_caccae_ATCC_43185.xml')
# repair_model(B_caccae)
# B_cell = read_sbml_model('/home/laurence/ME/data/microbiome/Agora_1_02_Western/sbml/Bacteroides_cellulosilyticus_DSM_14838.xml')
# repair_model(B_cell)
# R_torques = read_sbml_model('/home/laurence/ME/data/microbiome/Agora_1_02_Western/sbml/Ruminococcus_torques_ATCC_27756.xml')
# repair_model(R_torques)
# C_spiro = read_sbml_model('/home/laurence/ME/data/microbiome/Agora_1_02_Western/sbml/Clostridium_spiroforme_DSM_1552.xml')
# repair_model(C_spiro)
# models = [B_caccae, R_torques, C_spiro]

mu: before=0.337029885021. after=0.337029885021
mu: before=0.527200385031. after=0.527200385031
mu: before=0.300894735033. after=0.300894735033
mu: before=0.1048777824. after=0.1048777824
mu: before=0.239918094612. after=0.239918094612
mu: before=0.119252112661. after=0.119252112661
mu: before=0.445426858511. after=0.445426858511
mu: before=0.507100103943. after=0.507100103943
mu: before=0.351568777871. after=0.351568777871
mu: before=0.165967078669. after=0.165967078669
mu: before=0.348346607661. after=0.348346607661
mu: before=0.121469370222. after=0.121469370222
mu: before=0.296304193686. after=0.296304193686
mu: before=0.114862357703. after=0.114862357703
mu: before=0.291237838539. after=0.291237838539


In [3]:
for mdl in models:
    for rxn in mdl.reactions:
        if rxn.objective_coefficient != 0:
            print('mdl=%s. rxn=%s. objcoeff=%s' % (mdl.id, rxn.id, rxn.objective_coefficient))

mdl=Bacteroides_cellulosilyticus_DSM_14838. rxn=biomass536. objcoeff=1.0
mdl=Bacteroides_caccae_ATCC_43185. rxn=biomass536. objcoeff=1.0
mdl=Parabacteroides_distasonis_ATCC_8503. rxn=biomass345. objcoeff=1.0
mdl=Ruminococcus_torques_ATCC_27756. rxn=biomass525. objcoeff=1.0
mdl=Clostridium_scindens_ATCC_35704. rxn=biomass524. objcoeff=1.0
mdl=Collinsella_aerofaciens_ATCC_25986. rxn=biomass524. objcoeff=1.0
mdl=Bacteroides_thetaiotaomicron_VPI_5482. rxn=biomass345. objcoeff=1.0
mdl=Bacteroides_vulgatus_ATCC_8482. rxn=biomass345. objcoeff=1.0
mdl=Bacteroides_ovatus_ATCC_8483. rxn=biomass536. objcoeff=1.0
mdl=Bacteroides_uniformis_ATCC_8492. rxn=biomass536. objcoeff=1.0
mdl=Eubacterium_rectale_ATCC_33656. rxn=biomass205. objcoeff=1.0
mdl=Clostridium_spiroforme_DSM_1552. rxn=biomass525. objcoeff=1.0
mdl=Faecalibacterium_prausnitzii_L2_6. rxn=biomass525. objcoeff=1.0
mdl=Ruminococcus_obeum_A2_162. rxn=biomass525. objcoeff=1.0
mdl=Dorea_longicatena_DSM_13814. rxn=biomass525. objcoeff=1.0


In [4]:
from cobra.flux_analysis import flux_variability_analysis
from six import iteritems

def exchanged_mets(mdl, cons_thresh=-0.1, secr_thresh=0.1, f_opt=1., reqd_thresh=-1e-8,
                  exclude=['biomass_c','h2o_e','pi_e','ppi_e','h_e','k_e']):
    ex_rxn_all = mdl.reactions.query('^EX_')
    ex_rxn = [rxn for rxn in ex_rxn_all if not any([m in [met.id for met in rxn.metabolites.keys()] for m in exclude]) ]
    
    rxn_met_dict = {rxn.id:[m for m,s in iteritems(rxn.metabolites) if s<0] for rxn in ex_rxn}
    met_rxn_dict = {m.id:rxn.id for rxn in ex_rxn for m,s in iteritems(rxn.metabolites) if s<0}
    
    sol = flux_variability_analysis(mdl, ex_rxn, solver='gurobi', fraction_of_optimum=f_opt)
    secr = [m.id for k,v in iteritems(sol) if v['maximum']>=secr_thresh for m in rxn_met_dict[k]]
    cons = [m.id for k,v in iteritems(sol) if v['minimum']<=cons_thresh for m in rxn_met_dict[k]]
    reqd = [m.id for k,v in iteritems(sol) if v['maximum']<=reqd_thresh for m in rxn_met_dict[k]]
    req_rxns = [k for k,v in iteritems(sol) if v['maximum']<=reqd_thresh]
    return {'secreted':secr, 'consumed':cons, 'required':reqd, 'met_rxn':met_rxn_dict, 'req_rxns':req_rxns}

# Data frames

### Sources

In [5]:
rows = []
# Cross-feed sources
for mdl in models:
    exmets = exchanged_mets(mdl)
    mr_dict= exmets['met_rxn']
    secr = exmets['secreted']
    reqd = exmets['required']
    for met in secr:
        rxn = mr_dict[met]
        rows.append({'met':met, 'source':mdl.id, 'rxn':rxn, 'node':np.nan, 'vmax':np.nan})
        
# Primary sources. Each source should provide all mets to be constrained

#ex1 = exchanged_mets(R_torques)
ex1 = exchanged_mets(models[0])
for met in ex1['consumed']:
    rows.append({'met':met, 'source':'primary', 'rxn':np.nan, 'node':3, 'vmax':1})
    rows.append({'met':met, 'source':'primary', 'rxn':np.nan, 'node':14, 'vmax':1})

df_source = pd.DataFrame(rows)
df_source.head(2)

,met,node,rxn,source,vmax
0,ppa_e,NaN,EX_ppa_e,Bacteroides_cellulosilyticus_DSM_14838,NaN
1,for_e,NaN,EX_for_e,Bacteroides_cellulosilyticus_DSM_14838,NaN


### Consumers

In [6]:
rows = []
for mdl in models:
    exmets = exchanged_mets(mdl)
    mr_dict= exmets['met_rxn']
    cons = exmets['consumed']
    for met in cons:
        rxn = mr_dict[met]
        rows.append({'met':met, 'id':mdl.id, 'rxn':rxn})

df_consumer = pd.DataFrame(rows)
df_consumer.head(2)

,id,met,rxn
0,Bacteroides_cellulosilyticus_DSM_14838,leu_L_e,EX_leu_L_e
1,Bacteroides_cellulosilyticus_DSM_14838,fuc_L_e,EX_fuc_L_e


## Keep only the mets that are shared across producers and consumers
### Only keep primary sources that feed one microbe to force cross-feeding

In [7]:
r_cons = df_consumer.met[ df_consumer.id=='Ruminococcus_torques_ATCC_27756']
b_cons = df_consumer.met[ df_consumer.id=='Bacteroides_caccae_ATCC_43185']
mets_r = [r for r in r_cons if r not in b_cons.values]
mets_r

['glyc_e', 'glu_L_e', 'gly_e', 'gln_L_e', 'asp_L_e']

In [8]:
df_primary_lim = df_source[
    (df_source['source']=='primary') &
    (df_source.met.isin(mets_r))
]
df_cross =  df_source[ df_source['source'] != 'primary']
df_source_lim = pd.concat([df_primary_lim, df_cross])

In [9]:
shared_mets = np.intersect1d(df_source.met, df_consumer.met)
shared_mets

array(['2obut_e', 'ac_e', 'acgam_e', 'adn_e', 'ala_D_e', 'ala_L_e',
       'arab_L_e', 'arg_L_e', 'asn_L_e', 'asp_L_e', 'cys_L_e', 'cytd_e',
       'dad_2_e', 'dgsn_e', 'for_e', 'fru_e', 'fuc_L_e', 'fum_e', 'gal_e',
       'gam_e', 'glc_D_e', 'gln_L_e', 'glu_L_e', 'gly_e', 'gsn_e',
       'gthox_e', 'gthrd_e', 'h2_e', 'leu_L_e', 'lys_L_e', 'man_e',
       'no2_e', 'orn_e', 'phe_L_e', 'pro_L_e', 'rib_D_e', 'ribflv_e',
       'rmn_e', 'ser_L_e', 'thr_L_e', 'tyr_L_e', 'val_L_e', 'xyl_D_e'],
      dtype=object)

In [10]:
df_consumer_shared = df_consumer[ df_consumer.met.isin(shared_mets)]
df_source_shared = df_source[ df_source.met.isin(shared_mets)]

### Unless required or provided by a source (including cross-feed), disable uptake

In [11]:
src_mets = set(df_source['met'].values)
for mdl in models:
    res = exchanged_mets(mdl, f_opt=0.5)    
    mr_dict = res['met_rxn']
    src_rxns = [mr_dict[m] for m in src_mets if m in mr_dict]
    req_rxns = res['req_rxns']
    #dfi = df_consumer_shared[ df_consumer_shared['id']==mdl.id]
    dfi = df_consumer[ df_consumer['id']==mdl.id]
    cons_rxns = dfi.rxn.values
    ### Need to have full set for at least one sol
    mdl.optimize()
    opt_rxns = [r.id for r in mdl.reactions.query('EX_') if r.x < -1e-6]
    
    for rxn in mdl.reactions.query('^EX_'):
        #if rxn.id not in req_rxns and rxn.id not in src_rxns:            
        if rxn.id not in req_rxns and rxn.id not in cons_rxns and rxn.id not in opt_rxns:
            rxn.lower_bound = 0.
            #print("%s. Set lb=0 for %s" % (mdl.id, rxn.id))
    # Ensure model can still grow
    mdl.optimize()
    if mdl.solution is None:
        print("%s cannot grow anymore!" % mdl.id)
    else:
        print("%s can still grow at %s" % (mdl.id, mdl.solution.f))

Bacteroides_cellulosilyticus_DSM_14838 can still grow at 0.336363722645
Bacteroides_caccae_ATCC_43185 can still grow at 0.52720038503
Parabacteroides_distasonis_ATCC_8503 can still grow at 0.300894735032
Ruminococcus_torques_ATCC_27756 can still grow at 0.104877782401
Clostridium_scindens_ATCC_35704 can still grow at 0.239918094613
Collinsella_aerofaciens_ATCC_25986 can still grow at 0.119252112658
Bacteroides_thetaiotaomicron_VPI_5482 can still grow at 0.445426858512
Bacteroides_vulgatus_ATCC_8482 can still grow at 0.507100103947
Bacteroides_ovatus_ATCC_8483 can still grow at 0.350898202922
Bacteroides_uniformis_ATCC_8492 can still grow at 0.165967078669
Eubacterium_rectale_ATCC_33656 can still grow at 0.348346607661
Clostridium_spiroforme_DSM_1552 can still grow at 0.121469370131
Faecalibacterium_prausnitzii_L2_6 can still grow at 0.296304193707
Ruminococcus_obeum_A2_162 can still grow at 0.114862357706
Dorea_longicatena_DSM_13814 can still grow at 0.29123783854


### Nodes

In [12]:
data = {'x':[0,1,2, 0,1,2, 0,1,2, 0,1,2, 0,1,2],
        'y':[0,0,0, 1,1,1, 2,2,2, 3,3,3, 4,4,4],
        'node':[0,1,2, 3,4,5, 6,7,8, 9,10,11, 12,13,14],
        'type':['org','org','org', 'source','org','org', 'org','org','org', 'org','org','org',
               'org','org','source'],
        'Tf':[2,2,2, 2,2,2, 2,2,2, 2,2,2, 2,2,2]
}
df_node = pd.DataFrame.from_dict(data)
df_node.index = df_node.node
df_node.head(2)

,Tf,node,type,x,y
node,,,,,
0,2,0,org,0,0
1,2,1,org,1,0


### Distances

In [13]:
rows = []
nodes = df_node.node.unique()
for i,rowi in df_node.iterrows():
    xi = rowi['x']
    yi = rowi['y']
    for j,rowj in df_node.iterrows():        
        xj = rowj['x']
        yj = rowj['y']
        dx = xj-xi
        dy = yj-yi
        d = np.sqrt(dx**2 + dy**2)
        rows.append({'i':i, 'j':j, 'd':d})

df_distance = pd.DataFrame(rows)
df_distance.head(2)

,d,i,j
0,0.0,0,0
1,1.0,0,1


### Organisms

In [14]:
rows = []
for mdl in models:
    for rxn in mdl.reactions:
        if rxn.objective_coefficient != 0:
            rows.append({'X0':0.9, 'biomass':rxn.id, 'id':mdl.id})  # if X0 = 1, allows to occupy a node without growing
            print('mdl=%s. rxn=%s. objcoeff=%s' % (mdl.id, rxn.id, rxn.objective_coefficient))
df_organism = pd.DataFrame(rows)
df_organism

mdl=Bacteroides_cellulosilyticus_DSM_14838. rxn=biomass536. objcoeff=1.0
mdl=Bacteroides_caccae_ATCC_43185. rxn=biomass536. objcoeff=1.0
mdl=Parabacteroides_distasonis_ATCC_8503. rxn=biomass345. objcoeff=1.0
mdl=Ruminococcus_torques_ATCC_27756. rxn=biomass525. objcoeff=1.0
mdl=Clostridium_scindens_ATCC_35704. rxn=biomass524. objcoeff=1.0
mdl=Collinsella_aerofaciens_ATCC_25986. rxn=biomass524. objcoeff=1.0
mdl=Bacteroides_thetaiotaomicron_VPI_5482. rxn=biomass345. objcoeff=1.0
mdl=Bacteroides_vulgatus_ATCC_8482. rxn=biomass345. objcoeff=1.0
mdl=Bacteroides_ovatus_ATCC_8483. rxn=biomass536. objcoeff=1.0
mdl=Bacteroides_uniformis_ATCC_8492. rxn=biomass536. objcoeff=1.0
mdl=Eubacterium_rectale_ATCC_33656. rxn=biomass205. objcoeff=1.0
mdl=Clostridium_spiroforme_DSM_1552. rxn=biomass525. objcoeff=1.0
mdl=Faecalibacterium_prausnitzii_L2_6. rxn=biomass525. objcoeff=1.0
mdl=Ruminococcus_obeum_A2_162. rxn=biomass525. objcoeff=1.0
mdl=Dorea_longicatena_DSM_13814. rxn=biomass525. objcoeff=1.0


,X0,biomass,id
0,0.9,biomass536,Bacteroides_cellulosilyticus_DSM_14838
1,0.9,biomass536,Bacteroides_caccae_ATCC_43185
2,0.9,biomass345,Parabacteroides_distasonis_ATCC_8503
3,0.9,biomass525,Ruminococcus_torques_ATCC_27756
4,0.9,biomass524,Clostridium_scindens_ATCC_35704
5,0.9,biomass524,Collinsella_aerofaciens_ATCC_25986
6,0.9,biomass345,Bacteroides_thetaiotaomicron_VPI_5482
7,0.9,biomass345,Bacteroides_vulgatus_ATCC_8482
8,0.9,biomass536,Bacteroides_ovatus_ATCC_8483
9,0.9,biomass536,Bacteroides_uniformis_ATCC_8492


# Solve

In [15]:
from dynamicme.location import Locater

locater = Locater()
#locater.create_problem(models, df_source_lim, df_consumer_shared, df_node, df_distance, df_organism)
locater.create_problem(models, df_source, df_consumer, df_node, df_distance, df_organism)

cobra/core/Model.py:186 UserWarning: Use add_reaction for single reactions


In [ ]:
from cobra.solvers import gurobi_solver

model = gurobi_solver.create_problem(locater.model)
model.Params.OutputFlag = 1
model.ModelSense == GRB.MAXIMIZE

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1


True

In [ ]:
model.optimize()

Optimize a model with 653989 rows, 229567 columns and 1742585 nonzeros
Variable types: 229372 continuous, 195 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-05, 2e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-05, 1e+03]
  RHS range        [9e-01, 1e+00]
Presolve removed 587399 rows and 192205 columns
Presolve time: 4.72s
Presolved: 66590 rows, 37362 columns, 273390 nonzeros
Variable types: 37258 continuous, 104 integer (104 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6057022e+02   2.584424e+06   8.380487e+10      7s
   37921    4.0584731e+01   2.855882e+03   5.157007e+08     10s
   43581    4.5263856e+01   2.776186e+03   3.398265e+09     15s
   49074    2.0901130e+01   1.301005e+03   2.420093e+09     20s
   57620    3.6380734e+00   1.650299e+02   5.638499e+08     25s
   65456    9.5749234e-01   1.015871e-01   1.585402e+12     30s
   68884    9.9725025e-01   0.000000e+00   5.279928e+04    

In [21]:
sol = gurobi_solver.format_solution(model, locater.model)

In [22]:
locater.model.solution = sol

In [23]:
for rxn in locater.model.reactions.query('^y_'):
    x = sol.x_dict[rxn.id]
    print("%-20.18s%-20s" % (x, rxn.id))

TypeError: 'NoneType' object has no attribute '__getitem__'

In [24]:
x_dict = locater.model.solution.x_dict

for cons in locater.model.metabolites.query('uptake_cap'):
    print('-'*60)
    print(cons.id)
    for rxn in cons.reactions:
        x = x_dict[rxn.id]
        print("%s\t%s\t%s" % (rxn.id, rxn.metabolites[cons], x))

------------------------------------------------------------
uptake_cap_Bacteroides_cellulosilyticus_DSM_14838_0_leu_L_e


TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
x_dict = locater.model.solution.x_dict

for cons in locater.model.metabolites.query('source_cap'):
    print('-'*60)
    print(cons.id)
    for rxn in cons.reactions:
        x = x_dict[rxn.id]
        print("%s\t%s\t%s" % (rxn.id, rxn.metabolites[cons], x))

In [ ]:
x_dict = locater.model.solution.x_dict

cons=locater.model.metabolites.query('crossfeed_')[0]
for rxn in cons.reactions:
    x = x_dict[rxn.id]
    print("%s\t%s\t%s" % (rxn.id, rxn.metabolites[cons], x))

## Check to see what each org,node is growing on

locater.model.reactions.query('^biomass')

locater.model.reactions.query('^y_')

cons = locater.model.metabolites.query('cons_uptake')[0]
cons.reactions

locater.model.metabolites.query('cons_uptake')

df_ex

df_sources.head()

# Plot the result

### Source lines

In [ ]:
ZERO = 1e-8
x_dict = locater.model.solution.x_dict
org_ids= [mdl.id for mdl in models]
nodes  = df_node.node.unique()
mets   = df_source.met.unique()

rows = []
line_rows = []
for ni,nrow in df_node.iterrows():
    x = nrow['x']
    y = nrow['y']
    node = nrow['node']
    typ = nrow['type']
    if typ=='source':
        density = df_source[ df_source['node']==node].vmax.iloc[0]
        rows.append({'x':x, 'y':y, 'density':density, 'type':typ})
        # rows.append({'x':x, 'y':y, 'density':0, 'type':typ})
    else:
        for oi,orow in df_organism.iterrows():
            mu_id = orow['biomass']
            org_id = orow['id']
            yid = 'y_%s_%s'%(org_id,node)
            yval = x_dict[yid]
            if yval > 0.5:
                mu_id = '%s_%s_%s'%(mu_id,org_id,node)
                mu = x_dict[mu_id]
            else:
                mu = 0.
            rows.append({'x':x,'y':y,'density':mu, 'type':org_id})
    ### Source lines
    for locp in nodes:
        xp = df_node.loc[locp]['x']
        yp = df_node.loc[locp]['y']
        for met in mets:
            sid = 's_%s_%s_%s'%(locp,node,met)            
            if x_dict.has_key(sid):
                spjl = x_dict[sid]
                if spjl>ZERO:
                    line_rows.append({'xj':x,'yj':y, 'xp':xp,'yp':yp, 's':spjl,'p':locp,'j':node,
                                     'type':'source', 'met':met})
            eid = 'e_%s_%s_%s'%(locp,node,met)
            if x_dict.has_key(eid):
                eijl = x_dict[eid]
                if eijl>ZERO:
                    line_rows.append({'xj':x,'yj':y, 'xp':xp,'yp':yp, 's':eijl,'p':locp,'j':node,
                                     'type':'cross', 'met':met})
dsplot = pd.DataFrame(rows)

In [ ]:
dsplot = dsplot.groupby(['x','y']).agg({'density':lambda x: max(x)}).reset_index()

In [ ]:
### Greater contrast for visualization
dsplot.loc[:,'z'] = dsplot.density
dsplot.loc[ dsplot.z==0,'z'] = -1

In [ ]:
df_line = pd.DataFrame(line_rows)

# Plot results

plot_dict = {
    'x':[0,2,0, 1,2,1, 0,1,2],
    'y':[1,1,0, 0,0,1, 2,2,2],
    'type':['cellb','acgal','org', 'org','org','org', 'org','org','org'],
    'density':[2,2,1, 1,1,1, 1,1,1]
}
dsplot = pd.DataFrame.from_dict(plot_dict)

from dynamicme.location import CommunityPlotter
plotter = CommunityPlotter()
plotter.plot(dsplot, zcol='density', points=True)

In [ ]:
import matplotlib
from scipy import interpolate

nres = 121
x = dsplot.x
y = dsplot.y
#z = dsplot.density
z = dsplot.z - dsplot.z.min()
xx = np.linspace(x.min(), x.max(), nres)
yy = np.linspace(y.min(), y.max(), nres)
X,Y = np.meshgrid(xx,yy)

### Add some noise to prevent singular matrix in interpolate
dr = 0 #0.001
xr = x + np.random.uniform(-dr, dr, size=x.shape)
yr = y + np.random.uniform(-dr, dr, size=y.shape)
#rbf = interpolate.Rbf(xr,yr,z, function='gaussian')
rbf = interpolate.Rbf(xr,yr,z, function='linear')
#rbf = interpolate.Rbf(x,y,z, function='cubic')
#rbf = interpolate.RectBivariateSpline(x,y,z)
Z = rbf(X,Y)

ax = plt.imshow(Z, vmin=z.min(), vmax=z.max(), origin='lower',
          extent=[x.min(), x.max(), y.min(), y.max()])

cmap = matplotlib.cm.get_cmap('nipy_spectral')
#cmap = matplotlib.cm.get_cmap('cubehelix')
#cmap = matplotlib.cm.get_cmap('YlGnBu_r')
#cmap = matplotlib.cm.get_cmap('gist_ncar')
#cmap = matplotlib.cm.get_cmap('terrain_r')
ax.set_cmap(cmap)

ax = plt.scatter(x, y, c=z, edgecolors='#ffffff', s=20+np.sqrt(z*10000), zorder=4, alpha=0.75)
ax.set_cmap(cmap)

### Plot the lines
for i,row in df_line.iterrows():
    xp = row['xp']
    yp = row['yp']
    xj = row['xj']
    yj = row['yj']
    s  = row['s']
    t  = row['type']
    #ax.axes.plot(x=[xp,xj], y=[yp,yj], color='#ffffff', lw=10*s)
    #plt.plot([xp,xj], [yp,yj], color='#aaaaaa', lw=(50*s)**0.5, ls='-', alpha=0.5)    
    dx = xj-xp
    dy = yj-yp
    w = 0.5+(5*s)**0.5
    color = '#cccccc' if t=='source' else '#ff6633'
    alpha = 0.9 #if t=='source' else 0.9
    zorder = 2 if t=='source' else 3
    plt.arrow(xp, yp, dx, dy, color=color, lw=w, ls='-', head_width=0.07,
              alpha=alpha, length_includes_head=True, zorder=zorder)

ax.axes.set_facecolor('#222222')
#ax.axes.set_facecolor('#000000')
plt.colorbar()
ax.figure.set_size_inches(4,8)

## TODO: color by type (organism/food source)

### Debug zone

### do source lines sum up correctly?
$$
\sum_j s_{ijl} \leq s^\mathrm{max}_{il}, \quad i \in \mathrm{Sources}, l \in Shared \qquad (6)
$$

In [ ]:
mets = df_source.met.unique()
for met in mets:
    svars = locater.model.reactions.query('^s_.*_%s'%met)
    svals = [x_dict[s.id] for s in svars]
    ssum  = sum(svals)
    print('%s:\tssum=%s'%(met,ssum))

[(s.id, x_dict[s.id]) for s in svars]

for rxn in locater.model.reactions.query('^y.*_1'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

### do cross-feed lines sum up correctly?
$$
\sum_j e_{pjl} \leq \sum_k v^{ex,k}_{pl}, \quad p \in Nodes, l \in Shared
$$

In [ ]:
ZERO = 1e-9
for met in mets:
    for locp in nodes:
        evars = []
        for j in nodes:
            eid = 'e_%s_%s_%s'%(locp,j,met)
            if locater.model.reactions.has_id(eid):
                e = locater.model.reactions.get_by_id(eid)
                evars.append(e)        
        if len(evars)>0:
            evals = [x_dict[e.id] for e in evars]
            esum = sum(evals)
            # Who is providing it?
            df_cross = df_source[ (df_source.met==met) & (df_source.source != 'primary')]            
            
            vexs = []
            vex_dict = {}
            y_dict = {}
            mu_dict = {}
            for i,crow in df_cross.iterrows():
                ex_id = crow['rxn']
                org   = crow['source']
                rxn_id = '%s_%s_%s'%(ex_id,org,locp)
                vex = x_dict[rxn_id]
                vexs.append(vex)                
                if abs(vex)>ZERO:
                    vex_dict[rxn_id] = vex
                    yid = 'y_%s_%s'%(org,locp)
                    y = x_dict[yid]
                    y_dict[yid] = y
                    mu_id = "%s_%s_%s"%(df_organism[ df_organism.id==org]['biomass'].iloc[0], org, locp)
                    mu = x_dict[mu_id]
                    mu_dict[mu_id] = mu
            vexsum = sum(vexs)
            if abs(vexsum)>ZERO or abs(esum)>ZERO:
                print("-"*50)
                print("%s. locp=%s"%(met,locp))
                print("esum (%s) =?= vex_sum (%s): %s" % (esum, vexsum, abs(esum-vexsum)<ZERO))
                print("vex_dict: %s"%vex_dict)
                print("y_dict: %s"%y_dict)
                print("mu_dict: %s"%mu_dict)

for rxn in locater.model.reactions.query('^y.*_6'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^s.*_12'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^e.*_0'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^e.*_12'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^e_0_.*'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^e_4_.*'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^e_8_.*'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))

for rxn in locater.model.reactions.query('^s.*_6'):
    x = x_dict[rxn.id]
    print('%-20.18s%s'%(x, rxn.id))